<a href="https://colab.research.google.com/github/Praxis-QR/RDWH/blob/main/PG24_2A_SQL42.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Yantrajaal Banner](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

![PostgreSQL 42](https://raw.githubusercontent.com/Praxis-QR/RDWH/main/images/PostgreSQL_24.01.png)<br>
[SlideDeck](https://docs.google.com/presentation/d/1hdkAGbMxLfO1GYe7RzEo3LhGZhsXkyjIfBqkA72B6LM/edit?usp=sharing)

In [1]:
from datetime import datetime
import pytz
print('ॐ श्री सरस्वत्यै नमः',datetime.now(pytz.timezone('Asia/Calcutta')))
!python --version
!lsb_release -a

ॐ श्री सरस्वत्यै नमः 2024-11-05 14:24:49.034570+05:30
Python 3.10.12
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.3 LTS
Release:	22.04
Codename:	jammy


#Install & Configure Postgres

For Connecting to a Postgres server hosted in the cloud Neon DB, see [this notebook](https://github.com/Praxis-QR/RDWH/blob/main/PG24_1B_SQL_CloudLab.ipynb)

In [2]:
# Install postgresql server

!apt update > /dev/null
!apt install postgresql > /dev/null
#!pip install psycopg2
import psycopg2
import pandas as pd
#!sudo -u postgres psql -V
!psql -V
!service postgresql start



W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


psql (PostgreSQL) 14.13 (Ubuntu 14.13-0ubuntu0.22.04.1)
 * Starting PostgreSQL 14 database server
   ...done.


In [3]:
# The installation creates Ubuntu user 'postgres' AND a Postgres database user 'postgres' both of which are superusers
#!cat /etc/passwd
# Use these users to create a new database user called 'upraxis' and give it a password 'upass'
#!sudo -u postgres createuser --superuser upraxis
!sudo -u postgres createuser upraxis
!sudo -u postgres psql -U postgres -c "ALTER USER upraxis PASSWORD 'upass';"
#
# Change the database superuser 'postgres' to a non-default value 'pass'
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'pass';"
#
# Use ubuntu superuser to create database dbpraxis and set its owner to upraxis
!sudo -u postgres createdb -O upraxis dbpraxis
#
# Create file to store the password. Else will be prompted each time
#
!echo "localhost:5432:dbpraxis:upraxis:upass" > ~/.pgpass
!echo "localhost:5432:dbpraxis:postgres:pass" >> ~/.pgpass
!chmod 0600 ~/.pgpass
!cat ~/.pgpass


ALTER ROLE
ALTER ROLE
localhost:5432:dbpraxis:upraxis:upass
localhost:5432:dbpraxis:postgres:pass


## Create and Load Tables

In [4]:
# Create Two tables using command line interface
#
!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c \
"drop table if exists Dept;\
create table Dept (\
    DeptID char(2),\
    DeptName varchar(20),\
    ManagerID char(6),\
    Location varchar(10),\
    LocationID char(1)\
    );\
insert into Dept (DeptID, DeptName, ManagerID, Location,LocationID) VALUES \
	('10', 'Corporate', '299034', 'Calcutta','1'), \
	('20', 'Sales', '239456', 'Calcutta','1'), \
	('30', 'Accounts', '822134', 'Calcutta','1'), \
	('40', 'Production', '349870', 'Bombay','2'),\
    ('60', 'Research', '546224', 'Delhi','3');\
"

NOTICE:  table "dept" does not exist, skipping
INSERT 0 5


In [5]:
!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c \
"drop table if exists Empl;\
create table Empl \
    (EmpID char(6),\
    LastName varchar(20),\
    FirstName varchar(20),\
    JobDesc varchar(10),\
    JoinDate date,\
    Salary int,\
    Comm decimal(3,2),\
    DeptID char(2),\
    HiredBy char(6)\
    );\
INSERT INTO Empl (EmpID, LastName, FirstName, JobDesc, JoinDate, Salary, Comm, DeptID, HiredBy) \
    VALUES \
	('742866', 'Bacchan', 'Amitabh', 'Executive', '2003-03-10', 50000, 0.1, '10', NULL), \
	('349870', 'Mukherjee', 'Rani', 'Manager', '2005-05-04', 25000, 0.06, '40', '742866'), \
	('865477', 'Dikshit', 'Madhuri', 'Clerk', '2002-04-04', 10000, 0.02, '20', '239456'), \
	('239456', 'Khan', 'Shahrukh', 'Manager', '2004-01-03', 30000, 0.07, '20', '742866'), \
	('897889', 'Sehwag', 'Virender', 'Cus_Rep', '2005-01-02', 15000, 0.05, '20', '239456'), \
	('123980', 'Dhoni', 'Mahender', 'Clerk', '2004-10-09', 9000, 0.02, '40', '349870'), \
	('822134', 'Dravid', 'Rahul', 'Sr Manager', '2000-06-04', 40000, 0.08, '30', '742866'), \
	('997445', 'Dalmia', 'Jagmohan', 'Clerk', '2001-07-01', 12000, 0.02, '30', '822134'), \
	('989007', 'Ganguly', 'Sourav', 'Cus_Rep', '2002-01-01', 20000, 0.03, '40', '299034'), \
	('299034', 'Ganesan', 'Rekha', 'Director', '2002-10-10', 60000, 0.11, '10', '742866'), \
	('546223', 'Karthikeyan', 'Narayan', 'Secretary', '2005-12-04', 40000, 0.09, '10', '299034'), \
    ('546224', 'Sunil', 'Chettri', 'Dy Manager', '2006-12-07', 40000, 0.05, '70', '989007'), \
	('223112', 'Mirza', 'Sania', 'Cus_Rep', '2001-11-19', 25000, 0.04, '30', '822134');\
"

NOTICE:  table "empl" does not exist, skipping
INSERT 0 13


In [6]:
#!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "select * from dept;"
#!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c "select * from dept;"

In [7]:
#!psql -h localhost -p 5432 -U postgres -d praxisdb -c "select * from empl;"
#!psql -h localhost -p 5432 -U upraxis -d dbpraxis -c "select * from empl;"

## Python Access
https://www.tutorialspoint.com/python_data_access/python_postgresql_database_connection.htm

In [8]:
# Local Colab server

conn = psycopg2.connect(database="dbpraxis", user='upraxis', password='upass', host='127.0.0.1', port= '5432')
#Creating a cursor object using the cursor() method
cursor = conn.cursor()
#Executing an MYSQL function using the execute() method
cursor.execute("select version()")
# Fetch a single row using fetchone() method.
data = cursor.fetchone()
print("Connection established to: ",data)
#Closing the connection
conn.close()

Connection established to:  ('PostgreSQL 14.13 (Ubuntu 14.13-0ubuntu0.22.04.1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0, 64-bit',)


In [9]:
# function to execute SQL select statements with Postgres
#
def runSelect(s):
    #Set up connection

    conn = psycopg2.connect(database="dbpraxis", user='upraxis', password='upass', host='127.0.0.1', port= '5432')
    #Setting auto commit true
    conn.autocommit = True
    #Creating a cursor object using the cursor() method
    cursor = conn.cursor()
    #Retrieving data
    cursor.execute(s)
    #Fetching all rows from the table
    result = cursor.fetchall();
    #print(len(result))
    if (len(result)) > 0:                                       # result contains at least one row
        colnames = [desc[0] for desc in cursor.description]     # get column names
        #print(colnames)
        resultdf = pd.DataFrame(result)                         # convert list to dataframe
        resultdf.columns = colnames                             # add column names
        #return resultdf
        return resultdf.style.hide()                            # hide the index
    else:
        print('no records found')
        return
# function to execute SQL non-select statements with Postgres
#
def runCmd(s):
    #Create Connection
    conn = psycopg2.connect(database="dbpraxis", user='upraxis', password='upass', host='127.0.0.1', port= '5432')
    #Creating a cursor object using the cursor() method
    cursor = conn.cursor()
    try:
        cursor.execute(s)
        print('Command executed')
        conn.commit()
    except:
        print('Command failed')
    #Closing the connection
    conn.close()

#Select - getting started

In [10]:
#runSelect("select * from dept where deptid = '100'")
#runSelect("select * from dept where deptid = '10'")
runSelect("select * from dept")

deptid,deptname,managerid,location,locationid
10,Corporate,299034,Calcutta,1
20,Sales,239456,Calcutta,1
30,Accounts,822134,Calcutta,1
40,Production,349870,Bombay,2
60,Research,546224,Delhi,3


In [11]:
runSelect('select * from empl')

empid,lastname,firstname,jobdesc,joindate,salary,comm,deptid,hiredby
742866,Bacchan,Amitabh,Executive,2003-03-10,50000,0.10,10,None
349870,Mukherjee,Rani,Manager,2005-05-04,25000,0.06,40,742866
865477,Dikshit,Madhuri,Clerk,2002-04-04,10000,0.02,20,239456
239456,Khan,Shahrukh,Manager,2004-01-03,30000,0.07,20,742866
897889,Sehwag,Virender,Cus_Rep,2005-01-02,15000,0.05,20,239456
123980,Dhoni,Mahender,Clerk,2004-10-09,9000,0.02,40,349870
822134,Dravid,Rahul,Sr Manager,2000-06-04,40000,0.08,30,742866
997445,Dalmia,Jagmohan,Clerk,2001-07-01,12000,0.02,30,822134
989007,Ganguly,Sourav,Cus_Rep,2002-01-01,20000,0.03,40,299034
299034,Ganesan,Rekha,Director,2002-10-10,60000,0.11,10,742866


In [12]:
runSelect('select lastname, jobdesc, salary from empl')

lastname,jobdesc,salary
Bacchan,Executive,50000
Mukherjee,Manager,25000
Dikshit,Clerk,10000
Khan,Manager,30000
Sehwag,Cus_Rep,15000
Dhoni,Clerk,9000
Dravid,Sr Manager,40000
Dalmia,Clerk,12000
Ganguly,Cus_Rep,20000
Ganesan,Director,60000


In [13]:
runSelect('select distinct(jobdesc) from empl')

jobdesc
Director
Manager
Dy Manager
Executive
Secretary
Cus_Rep
Clerk
Sr Manager


#Where

## Equal, Not Equal, Greater, Less

In [14]:
runSelect("select firstname, lastname, jobdesc from empl where jobdesc = 'Clerk'")

firstname,lastname,jobdesc
Madhuri,Dikshit,Clerk
Mahender,Dhoni,Clerk
Jagmohan,Dalmia,Clerk


In [15]:
runSelect("select firstname, lastname, jobdesc from empl where jobdesc != 'Clerk'")

firstname,lastname,jobdesc
Amitabh,Bacchan,Executive
Rani,Mukherjee,Manager
Shahrukh,Khan,Manager
Virender,Sehwag,Cus_Rep
Rahul,Dravid,Sr Manager
Sourav,Ganguly,Cus_Rep
Rekha,Ganesan,Director
Narayan,Karthikeyan,Secretary
Chettri,Sunil,Dy Manager
Sania,Mirza,Cus_Rep


In [16]:
runSelect("select firstname, lastname, salary from empl where salary > 35000")

firstname,lastname,salary
Amitabh,Bacchan,50000
Rahul,Dravid,40000
Rekha,Ganesan,60000
Narayan,Karthikeyan,40000
Chettri,Sunil,40000


In [17]:
runSelect("select firstname, lastname, salary, joindate from empl where joindate < '2002-01-01'")

firstname,lastname,salary,joindate
Rahul,Dravid,40000,2000-06-04
Jagmohan,Dalmia,12000,2001-07-01
Sania,Mirza,25000,2001-11-19


## Like

In [18]:
runSelect("select firstname, lastname, salary from empl where lastname like 'D%'")

firstname,lastname,salary
Madhuri,Dikshit,10000
Mahender,Dhoni,9000
Rahul,Dravid,40000
Jagmohan,Dalmia,12000


In [19]:
runSelect("select firstname, lastname, joindate from empl where lastname like '____i_'")

firstname,lastname,joindate
Rahul,Dravid,2000-06-04
Jagmohan,Dalmia,2001-07-01


In [20]:
runSelect("select firstname, lastname, jobdesc from empl where lastname like '____i%'")

firstname,lastname,jobdesc
Mahender,Dhoni,Clerk
Rahul,Dravid,Sr Manager
Jagmohan,Dalmia,Clerk


## Between

In [21]:
runSelect("select firstname, lastname, salary from empl where salary between 10000 and 20000")

firstname,lastname,salary
Madhuri,Dikshit,10000
Virender,Sehwag,15000
Jagmohan,Dalmia,12000
Sourav,Ganguly,20000


In [22]:
runSelect("select firstname, lastname, joindate from empl where joindate between '2004-01-01' and '2005-01-01'")

firstname,lastname,joindate
Shahrukh,Khan,2004-01-03
Mahender,Dhoni,2004-10-09


In [23]:
runSelect("select firstname, lastname, joindate from empl where lastname between 'Dho' and 'Kha'")

firstname,lastname,joindate
Madhuri,Dikshit,2002-04-04
Mahender,Dhoni,2004-10-09
Rahul,Dravid,2000-06-04
Sourav,Ganguly,2002-01-01
Rekha,Ganesan,2002-10-10
Narayan,Karthikeyan,2005-12-04


## IN

In [24]:
runSelect("select firstname, lastname, deptid from empl where deptid in ('30','20','70')")

firstname,lastname,deptid
Madhuri,Dikshit,20
Shahrukh,Khan,20
Virender,Sehwag,20
Rahul,Dravid,30
Jagmohan,Dalmia,30
Chettri,Sunil,70
Sania,Mirza,30


In [25]:
runSelect("select firstname, lastname, salary from empl where firstname in ('Madhuri','Shahrukh','Virender')")

firstname,lastname,salary
Madhuri,Dikshit,10000
Shahrukh,Khan,30000
Virender,Sehwag,15000


## Order by

In [26]:
runSelect("select firstname, lastname, joindate from empl where firstname in ('Madhuri','Shahrukh','Virender') order by joindate")

firstname,lastname,joindate
Madhuri,Dikshit,2002-04-04
Shahrukh,Khan,2004-01-03
Virender,Sehwag,2005-01-02


In [27]:
runSelect("select firstname, lastname, jobdesc, salary, deptid from empl where deptid = '10' order by salary desc")

firstname,lastname,jobdesc,salary,deptid
Rekha,Ganesan,Director,60000,10
Amitabh,Bacchan,Executive,50000,10
Narayan,Karthikeyan,Secretary,40000,10


## Logical AND | OR

In [28]:
runSelect("select lastname, jobdesc, salary, deptid from empl where deptid = '20'")

lastname,jobdesc,salary,deptid
Dikshit,Clerk,10000,20
Khan,Manager,30000,20
Sehwag,Cus_Rep,15000,20


In [29]:
runSelect("select lastname, jobdesc, salary, deptid from empl where deptid = '20' and salary > 10000")

lastname,jobdesc,salary,deptid
Khan,Manager,30000,20
Sehwag,Cus_Rep,15000,20


In [30]:
runSelect("select lastname, jobdesc, salary, deptid from empl where deptid = '20' and salary > 10000 and jobdesc = 'Clerk'")

no records found


In [31]:
runSelect("select lastname, jobdesc, deptid from empl where deptid = '70'")

lastname,jobdesc,deptid
Sunil,Dy Manager,70


In [32]:
runSelect("select lastname, jobdesc, deptid from empl where deptid = '70' or jobdesc like '%Manager'")

lastname,jobdesc,deptid
Mukherjee,Manager,40
Khan,Manager,20
Dravid,Sr Manager,30
Sunil,Dy Manager,70


In [33]:
runSelect("select lastname, jobdesc, deptid from empl where deptid = '70' or jobdesc like '%Manager' or lastname = 'Dikshit'")

lastname,jobdesc,deptid
Mukherjee,Manager,40
Dikshit,Clerk,20
Khan,Manager,20
Dravid,Sr Manager,30
Sunil,Dy Manager,70


In [34]:
runSelect("select lastname, jobdesc, salary from empl where jobdesc = 'Cus_Rep' or jobdesc = 'Clerk' and salary < 16000")

lastname,jobdesc,salary
Dikshit,Clerk,10000
Sehwag,Cus_Rep,15000
Dhoni,Clerk,9000
Dalmia,Clerk,12000
Ganguly,Cus_Rep,20000
Mirza,Cus_Rep,25000


In [35]:
runSelect("select lastname, jobdesc, salary from empl where (jobdesc = 'Cus_Rep' or jobdesc = 'Clerk') and salary < 16000")

lastname,jobdesc,salary
Dikshit,Clerk,10000
Sehwag,Cus_Rep,15000
Dhoni,Clerk,9000
Dalmia,Clerk,12000


# Single Row Functions

In [36]:
runSelect("select lastname, deptid, salary, comm, salary+salary*comm payout from empl where deptid = '20'")

lastname,deptid,salary,comm,payout
Dikshit,20,10000,0.02,10200.00
Khan,20,30000,0.07,32100.00
Sehwag,20,15000,0.05,15750.00


In [37]:
runSelect("select lastname, firstname, deptid, substring(firstname,1,3) nick from empl where deptid = '20'")

lastname,firstname,deptid,nick
Dikshit,Madhuri,20,Mad
Khan,Shahrukh,20,Sha
Sehwag,Virender,20,Vir


In [38]:
runSelect("select lastname, firstname, deptid, concat('\"',substring(firstname,2,3),'\" ',lastname) nick from empl where deptid = '20'")

lastname,firstname,deptid,nick
Dikshit,Madhuri,20,"""adh"" Dikshit"
Khan,Shahrukh,20,"""hah"" Khan"
Sehwag,Virender,20,"""ire"" Sehwag"


In [39]:
runSelect("select LastName, Current_Date,JoinDate, to_char(JoinDate,'Day') JoinDay, to_char(JoinDate,'DD-mon-YY') JoinDate2, \
Current_Date-JoinDate DaysSince from Empl")

#https://blog.quest.com/exploring-postgres-date-formats-and-their-different-functions/


lastname,current_date,joindate,joinday,joindate2,dayssince
Bacchan,2024-11-05,2003-03-10,Monday,10-mar-03,7911
Mukherjee,2024-11-05,2005-05-04,Wednesday,04-may-05,7125
Dikshit,2024-11-05,2002-04-04,Thursday,04-apr-02,8251
Khan,2024-11-05,2004-01-03,Saturday,03-jan-04,7612
Sehwag,2024-11-05,2005-01-02,Sunday,02-jan-05,7247
Dhoni,2024-11-05,2004-10-09,Saturday,09-oct-04,7332
Dravid,2024-11-05,2000-06-04,Sunday,04-jun-00,8920
Dalmia,2024-11-05,2001-07-01,Sunday,01-jul-01,8528
Ganguly,2024-11-05,2002-01-01,Tuesday,01-jan-02,8344
Ganesan,2024-11-05,2002-10-10,Thursday,10-oct-02,8062


# MultiRow Function | Group By | Having
for more complex examples that demonstrate Grouping Sets, Cubes, Roll UP, Crosstabs and other OLAP related SQL see [this notebook](https://github.com/Praxis-QR/RDWH/blob/main/PostgreSQL_03B_SQL_Level2.ipynb)

In [40]:
runSelect("select sum(salary) Total_Salary from empl")

total_salary
376000


In [41]:
runSelect("select round(avg(salary),2) Average_Salary from empl")

average_salary
28923.08


In [42]:
runSelect("select max(salary) Max_Salary from empl")

max_salary
60000


In [43]:
runSelect("select deptid, sum(salary) Total, max(salary) Max from empl group by deptid")

deptid,total,max
10,150000,60000
40,54000,25000
70,40000,40000
20,55000,30000
30,77000,40000


In [44]:
runSelect("select deptid, sum(salary) Total, max(salary) Max from empl group by deptid having sum(salary) > 50000")

deptid,total,max
10,150000,60000
40,54000,25000
20,55000,30000
30,77000,40000


In [45]:
runSelect("select deptid, sum(salary) Total, max(salary) Max from empl group by deptid having sum(salary) > 50000 and max(salary) < 35000")

deptid,total,max
40,54000,25000
20,55000,30000


#Conditionals | CASE

In [46]:
runSelect("select lastname, jobdesc, salary, comm, salary+salary*comm Payout,\
case when jobdesc like '%Manager' then 2000 else 0 end bonus from empl")

lastname,jobdesc,salary,comm,payout,bonus
Bacchan,Executive,50000,0.10,55000.00,0
Mukherjee,Manager,25000,0.06,26500.00,2000
Dikshit,Clerk,10000,0.02,10200.00,0
Khan,Manager,30000,0.07,32100.00,2000
Sehwag,Cus_Rep,15000,0.05,15750.00,0
Dhoni,Clerk,9000,0.02,9180.00,0
Dravid,Sr Manager,40000,0.08,43200.00,2000
Dalmia,Clerk,12000,0.02,12240.00,0
Ganguly,Cus_Rep,20000,0.03,20600.00,0
Ganesan,Director,60000,0.11,66600.00,0


In [47]:
runSelect("select lastname, deptid,\
case \
when deptid in ('10','20') then 'Head' \
when deptid = '30' then 'Regional'\
else 'Branch' \
end \
 office from empl")

lastname,deptid,office
Bacchan,10,Head
Mukherjee,40,Branch
Dikshit,20,Head
Khan,20,Head
Sehwag,20,Head
Dhoni,40,Branch
Dravid,30,Regional
Dalmia,30,Regional
Ganguly,40,Branch
Ganesan,10,Head


#Join

## Basic Inner Join

In [48]:
sql = 'select firstname, lastname, salary, deptname, location, managerid from empl, dept where empl.deptid = dept.deptid'
runSelect(sql)

firstname,lastname,salary,deptname,location,managerid
Narayan,Karthikeyan,40000,Corporate,Calcutta,299034
Rekha,Ganesan,60000,Corporate,Calcutta,299034
Amitabh,Bacchan,50000,Corporate,Calcutta,299034
Virender,Sehwag,15000,Sales,Calcutta,239456
Shahrukh,Khan,30000,Sales,Calcutta,239456
Madhuri,Dikshit,10000,Sales,Calcutta,239456
Sania,Mirza,25000,Accounts,Calcutta,822134
Jagmohan,Dalmia,12000,Accounts,Calcutta,822134
Rahul,Dravid,40000,Accounts,Calcutta,822134
Sourav,Ganguly,20000,Production,Bombay,349870


In [49]:
sql = 'select empid, firstname, deptname, location from empl INNER JOIN dept ON empl.deptid = dept.deptid'
runSelect(sql)

empid,firstname,deptname,location
546223,Narayan,Corporate,Calcutta
299034,Rekha,Corporate,Calcutta
742866,Amitabh,Corporate,Calcutta
897889,Virender,Sales,Calcutta
239456,Shahrukh,Sales,Calcutta
865477,Madhuri,Sales,Calcutta
223112,Sania,Accounts,Calcutta
997445,Jagmohan,Accounts,Calcutta
822134,Rahul,Accounts,Calcutta
989007,Sourav,Production,Bombay


In [50]:
sql = 'select deptname,round(sum(salary),0) payroll from empl INNER JOIN dept ON empl.deptid = dept.deptid \
group by deptname'
runSelect(sql)

deptname,payroll
Corporate,150000
Accounts,77000
Sales,55000
Production,54000


## Outer Join

### Left Outer

In [51]:
sql = 'select empid, firstname, deptname, location from empl LEFT JOIN dept ON empl.deptid = dept.deptid'
runSelect(sql)

empid,firstname,deptname,location
546223,Narayan,Corporate,Calcutta
299034,Rekha,Corporate,Calcutta
742866,Amitabh,Corporate,Calcutta
897889,Virender,Sales,Calcutta
239456,Shahrukh,Sales,Calcutta
865477,Madhuri,Sales,Calcutta
223112,Sania,Accounts,Calcutta
997445,Jagmohan,Accounts,Calcutta
822134,Rahul,Accounts,Calcutta
989007,Sourav,Production,Bombay


### Right Outer

In [52]:
sql = 'select empid, firstname, deptname, location from empl RIGHT JOIN dept ON empl.deptid = dept.deptid'
runSelect(sql)

empid,firstname,deptname,location
546223,Narayan,Corporate,Calcutta
299034,Rekha,Corporate,Calcutta
742866,Amitabh,Corporate,Calcutta
897889,Virender,Sales,Calcutta
239456,Shahrukh,Sales,Calcutta
865477,Madhuri,Sales,Calcutta
223112,Sania,Accounts,Calcutta
997445,Jagmohan,Accounts,Calcutta
822134,Rahul,Accounts,Calcutta
989007,Sourav,Production,Bombay


### Full Outer

In [53]:
sql = 'select empid, firstname, deptname, location from empl FULL JOIN dept ON empl.deptid = dept.deptid'
runSelect(sql)

empid,firstname,deptname,location
546223,Narayan,Corporate,Calcutta
299034,Rekha,Corporate,Calcutta
742866,Amitabh,Corporate,Calcutta
897889,Virender,Sales,Calcutta
239456,Shahrukh,Sales,Calcutta
865477,Madhuri,Sales,Calcutta
223112,Sania,Accounts,Calcutta
997445,Jagmohan,Accounts,Calcutta
822134,Rahul,Accounts,Calcutta
989007,Sourav,Production,Bombay


In [54]:
sql = 'select deptname,round(sum(salary),0) payroll from empl FULL JOIN dept ON empl.deptid = dept.deptid \
group by deptname'
runSelect(sql)

deptname,payroll
None,40000
Research,None
Corporate,150000
Accounts,77000
Sales,55000
Production,54000


## Recursive Joins

### Three Tables

In [55]:
cmd = "select w.lastname , w.deptID, '>' linksTo  , d.deptID ,d.managerID,'<' linksTo, m.empID, m.lastname \
        from Empl w, Dept d, Empl m \
        where w.deptID = d.deptID   \
        and d.managerID = m.empID \
    "
runSelect(cmd)

lastname,deptid,linksto,deptid,managerid,linksto,empid,lastname
Karthikeyan,10,>,10,299034,<,299034,Ganesan
Ganesan,10,>,10,299034,<,299034,Ganesan
Bacchan,10,>,10,299034,<,299034,Ganesan
Sehwag,20,>,20,239456,<,239456,Khan
Khan,20,>,20,239456,<,239456,Khan
Dikshit,20,>,20,239456,<,239456,Khan
Mirza,30,>,30,822134,<,822134,Dravid
Dalmia,30,>,30,822134,<,822134,Dravid
Dravid,30,>,30,822134,<,822134,Dravid
Ganguly,40,>,40,349870,<,349870,Mukherjee


### Self Join / Single Table

In [56]:
cmd = "Select w.FirstName, w.JobDesc, w.DeptID, \
    ' hired by ' hiredby, \
    m.FirstName,m.JobDesc,m.DeptID \
    from empl w, empl m \
    where w.HiredBy = m.EmpID"

runSelect(cmd)

firstname,jobdesc,deptid,hiredby,firstname,jobdesc,deptid
Rani,Manager,40,hired by,Amitabh,Executive,10
Madhuri,Clerk,20,hired by,Shahrukh,Manager,20
Shahrukh,Manager,20,hired by,Amitabh,Executive,10
Virender,Cus_Rep,20,hired by,Shahrukh,Manager,20
Mahender,Clerk,40,hired by,Rani,Manager,40
Rahul,Sr Manager,30,hired by,Amitabh,Executive,10
Jagmohan,Clerk,30,hired by,Rahul,Sr Manager,30
Sourav,Cus_Rep,40,hired by,Rekha,Director,10
Rekha,Director,10,hired by,Amitabh,Executive,10
Narayan,Secretary,10,hired by,Rekha,Director,10


#Subquery

In [57]:
sql = "select lastname, salary, jobdesc, deptid from empl where \
salary = (select max(salary) from empl where deptid = '20') \
and deptid = '20'"

runSelect(sql)

lastname,salary,jobdesc,deptid
Khan,30000,Manager,20


In [58]:
sql = "select lastname, salary, jobdesc, deptid from empl where salary < (select avg(salary) from empl where deptid = '20')\
and deptid = '20'"
runSelect(sql)

lastname,salary,jobdesc,deptid
Dikshit,10000,Clerk,20
Sehwag,15000,Cus_Rep,20


## Exists | Not Exists

In [59]:
#!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c "\
sql = "\
select firstname, jobdesc, deptid from empl \
where exists \
(select * from dept where empl.deptid = dept.deptid)\
order by deptid \
"
runSelect(sql)

firstname,jobdesc,deptid
Amitabh,Executive,10
Rekha,Director,10
Narayan,Secretary,10
Madhuri,Clerk,20
Shahrukh,Manager,20
Virender,Cus_Rep,20
Sania,Cus_Rep,30
Rahul,Sr Manager,30
Jagmohan,Clerk,30
Sourav,Cus_Rep,40


In [60]:
#!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c "\
sql = "\
select firstname, jobdesc, deptid from empl \
where not exists \
(select * from dept where empl.deptid = dept.deptid)\
order by deptid \
"
runSelect(sql)

firstname,jobdesc,deptid
Chettri,Dy Manager,70


## ALL | ANY

In [61]:
#!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c "\
sql = "\
select firstname, jobdesc, deptid, salary from empl \
where salary > all \
(select avg(salary) from empl)\
"
runSelect(sql)

firstname,jobdesc,deptid,salary
Amitabh,Executive,10,50000
Shahrukh,Manager,20,30000
Rahul,Sr Manager,30,40000
Rekha,Director,10,60000
Narayan,Secretary,10,40000
Chettri,Dy Manager,70,40000


In [62]:
#!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c "\
sql = "\
select firstname, jobdesc, deptid, salary from empl \
where salary <= any \
(select min(salary) from empl)\
"
runSelect(sql)

firstname,jobdesc,deptid,salary
Mahender,Clerk,40,9000


In [63]:
#!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c "\
sql = "\
select firstname, jobdesc, deptid, salary from empl \
where salary >= any \
(select max(salary) from empl)\
"
runSelect(sql)

firstname,jobdesc,deptid,salary
Rekha,Director,10,60000


In [64]:
!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c "\
select firstname, jobdesc, deptid, salary from empl \
where deptid = ALL  \
(select deptid from dept where location = 'Calcutta' )\
"
#runSelect(sql)

 firstname | jobdesc | deptid | salary 
-----------+---------+--------+--------
(0 rows)



In [65]:
!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c "\
select firstname, jobdesc, deptid, salary from empl \
where deptid = any  \
(select deptid from dept where location = 'Calcutta' )\
"

 firstname |  jobdesc   | deptid | salary 
-----------+------------+--------+--------
 Amitabh   | Executive  | 10     |  50000
 Madhuri   | Clerk      | 20     |  10000
 Shahrukh  | Manager    | 20     |  30000
 Virender  | Cus_Rep    | 20     |  15000
 Rahul     | Sr Manager | 30     |  40000
 Jagmohan  | Clerk      | 30     |  12000
 Rekha     | Director   | 10     |  60000
 Narayan   | Secretary  | 10     |  40000
 Sania     | Cus_Rep    | 30     |  25000
(9 rows)



In [66]:
!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c "\
select firstname, jobdesc, salary from empl \
where salary > ANY  \
(select salary from empl where jobdesc like '%Manager%' )\
"

 firstname |  jobdesc   | salary 
-----------+------------+--------
 Amitabh   | Executive  |  50000
 Shahrukh  | Manager    |  30000
 Rahul     | Sr Manager |  40000
 Rekha     | Director   |  60000
 Narayan   | Secretary  |  40000
 Chettri   | Dy Manager |  40000
(6 rows)



In [67]:
!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c "\
select firstname, jobdesc, salary from empl \
where salary > ALL  \
(select salary from empl where jobdesc like '%Manager%' )\
"

 firstname |  jobdesc  | salary 
-----------+-----------+--------
 Amitabh   | Executive |  50000
 Rekha     | Director  |  60000
(2 rows)



## Common Table Extensions CTE

In [68]:
sql = "\
select jobdesc, avg(salary) avgsalary from empl group by jobdesc \
"
runSelect(sql)

jobdesc,avgsalary
Director,60000.000000000000
Manager,27500.000000000000
Dy Manager,40000.000000000000
Executive,50000.000000000000
Secretary,40000.000000000000
Cus_Rep,20000.000000000000
Clerk,10333.3333333333333333
Sr Manager,40000.000000000000


In [69]:
sql = "\
with MeanSalary as \
    (select jobdesc, round(avg(salary),0) avgsalary from empl group by jobdesc) \
select lastname, empl.jobdesc, salary, avgsalary from empl, meansalary \
where empl.jobdesc = meansalary.jobdesc \
order  by jobdesc \
"
runSelect(sql)

lastname,jobdesc,salary,avgsalary
Dalmia,Clerk,12000,10333
Dikshit,Clerk,10000,10333
Dhoni,Clerk,9000,10333
Sehwag,Cus_Rep,15000,20000
Mirza,Cus_Rep,25000,20000
Ganguly,Cus_Rep,20000,20000
Ganesan,Director,60000,60000
Sunil,Dy Manager,40000,40000
Bacchan,Executive,50000,50000
Khan,Manager,30000,27500


In [70]:
sql = "\
select lastname, empl.jobdesc, salary, avgsalary from \
empl, (select jobdesc, round(avg(salary),0) avgsalary from empl group by jobdesc) meansalary \
where empl.jobdesc = meansalary.jobdesc \
order  by jobdesc \
"
runSelect(sql)

lastname,jobdesc,salary,avgsalary
Dalmia,Clerk,12000,10333
Dikshit,Clerk,10000,10333
Dhoni,Clerk,9000,10333
Sehwag,Cus_Rep,15000,20000
Mirza,Cus_Rep,25000,20000
Ganguly,Cus_Rep,20000,20000
Ganesan,Director,60000,60000
Sunil,Dy Manager,40000,40000
Bacchan,Executive,50000,50000
Khan,Manager,30000,27500


### CTE Recursion

In [71]:
# recursion
sql = "\
with recursive hiring_chain as (\
select empid, lastname, hiredby \
    from empl where lastname = 'Ganesan' \
union all \
select \
e.empid, e.lastname,e.hiredby \
    from empl e \
    inner join hiring_chain hc on hc.empid = e.hiredby \
) \
select * from hiring_chain \
"

runSelect(sql)

empid,lastname,hiredby
299034,Ganesan,742866
989007,Ganguly,299034
546223,Karthikeyan,299034
546224,Sunil,989007


### More Sophisticated Recursion
note the switch between upward chain and downward chain in the JOIN column <br>


*   Downward > e.hiredby = hc.empid
*   Upward   > e.empid = hc.hiredby

Courtesy [ChatGPT](https://chat.openai.com/share/80e291a6-bbd2-42d8-ba3d-6e01be4eafd7)

In [72]:
# Downward Chain
#
sql = "\
with recursive hiring_chain as (\
select empid, lastname, hiredby, 1 as level \
    from empl where hiredby = '299034' \
union all \
select \
e.empid, e.lastname,e.hiredby, hc.level+1 \
    from empl e \
    inner join hiring_chain hc on e.hiredby = hc.empid  \
) \
select * from hiring_chain \
"

runSelect(sql)

empid,lastname,hiredby,level
989007,Ganguly,299034,1
546223,Karthikeyan,299034,1
546224,Sunil,989007,2


In [73]:
# Downward Chain - Style 2
#
sql = "\
with recursive hiring_chain as (\
select empid, lastname, hiredby, 1 as level \
    from empl where hiredby = (select empid from empl where lastname = 'Ganesan') \
union all \
select \
e.empid, e.lastname,e.hiredby, hc.level+1 \
    from empl e \
    inner join hiring_chain hc on e.hiredby = hc.empid  \
) \
select * from hiring_chain \
"

runSelect(sql)

empid,lastname,hiredby,level
989007,Ganguly,299034,1
546223,Karthikeyan,299034,1
546224,Sunil,989007,2


In [74]:
# Upward Chain
#
sql = "\
with recursive hiring_chain as (\
select empid, lastname, hiredby, 1 as level \
    from empl where lastname = 'Ganesan' \
union all \
select \
e.empid, e.lastname,e.hiredby, hc.level+1 \
    from empl e \
    inner join hiring_chain hc on e.empid = hc.hiredby  \
) \
select * from hiring_chain \
"

runSelect(sql)

empid,lastname,hiredby,level
299034,Ganesan,742866,1
742866,Bacchan,None,2


In [75]:
# Upward Chain
#
sql = "\
with recursive hiring_chain as (\
select empid, lastname, hiredby, 1 as level \
    from empl where lastname = 'Sunil' \
union all \
select \
e.empid, e.lastname,e.hiredby, hc.level+1 \
    from empl e \
    inner join hiring_chain hc on e.empid = hc.hiredby  \
) \
select * from hiring_chain \
"

runSelect(sql)

empid,lastname,hiredby,level
546224,Sunil,989007,1
989007,Ganguly,299034,2
299034,Ganesan,742866,3
742866,Bacchan,None,4


#Window Functions

In [76]:
sql = "select deptid, lastname, salary, \
    avg(salary) over (partition by deptid) \
    from empl"
runSelect(sql)

deptid,lastname,salary,avg
10,Bacchan,50000,50000.000000000000
10,Ganesan,60000,50000.000000000000
10,Karthikeyan,40000,50000.000000000000
20,Dikshit,10000,18333.333333333333
20,Khan,30000,18333.333333333333
20,Sehwag,15000,18333.333333333333
30,Mirza,25000,25666.666666666667
30,Dravid,40000,25666.666666666667
30,Dalmia,12000,25666.666666666667
40,Ganguly,20000,18000.000000000000


In [77]:
sql = "select deptname, lastname, salary, avg(salary) over (partition by deptname) from empl  inner join dept using(deptid)"
runSelect(sql)

deptname,lastname,salary,avg
Accounts,Dravid,40000,25666.666666666667
Accounts,Dalmia,12000,25666.666666666667
Accounts,Mirza,25000,25666.666666666667
Corporate,Karthikeyan,40000,50000.000000000000
Corporate,Ganesan,60000,50000.000000000000
Corporate,Bacchan,50000,50000.000000000000
Production,Ganguly,20000,18000.000000000000
Production,Dhoni,9000,18000.000000000000
Production,Mukherjee,25000,18000.000000000000
Sales,Dikshit,10000,18333.333333333333


In [78]:
sql = "select deptname, lastname, salary, \
    row_number() over (partition by deptname order by salary) \
    from empl  inner join dept using(deptid)"
runSelect(sql)

deptname,lastname,salary,row_number
Accounts,Dalmia,12000,1
Accounts,Mirza,25000,2
Accounts,Dravid,40000,3
Corporate,Karthikeyan,40000,1
Corporate,Bacchan,50000,2
Corporate,Ganesan,60000,3
Production,Dhoni,9000,1
Production,Ganguly,20000,2
Production,Mukherjee,25000,3
Sales,Dikshit,10000,1


In [79]:
sql = "select deptname, lastname, salary, \
        rank() over (partition by deptname order by salary desc) \
        from empl  inner join dept using(deptid)"
runSelect(sql)

deptname,lastname,salary,rank
Accounts,Dravid,40000,1
Accounts,Mirza,25000,2
Accounts,Dalmia,12000,3
Corporate,Ganesan,60000,1
Corporate,Bacchan,50000,2
Corporate,Karthikeyan,40000,3
Production,Mukherjee,25000,1
Production,Ganguly,20000,2
Production,Dhoni,9000,3
Sales,Khan,30000,1


In [80]:
sql = "select deptname, lastname, salary, \
first_value(salary) over (partition by deptname order by salary) as deptlow, \
last_value(salary) over (partition by deptname order by salary RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as depthigh \
from empl  inner join dept using(deptid)"
runSelect(sql)

deptname,lastname,salary,deptlow,depthigh
Accounts,Dalmia,12000,12000,40000
Accounts,Mirza,25000,12000,40000
Accounts,Dravid,40000,12000,40000
Corporate,Karthikeyan,40000,40000,60000
Corporate,Bacchan,50000,40000,60000
Corporate,Ganesan,60000,40000,60000
Production,Dhoni,9000,9000,25000
Production,Ganguly,20000,9000,25000
Production,Mukherjee,25000,9000,25000
Sales,Dikshit,10000,10000,30000


In [81]:
sql = "select deptname, lastname, salary, \
lag(salary,1) over (partition by deptname order by salary) as prev_emp, \
salary - lag(salary,1) over (partition by deptname order by salary) as gap_up \
from empl  inner join dept using(deptid)"
runSelect(sql)

deptname,lastname,salary,prev_emp,gap_up
Accounts,Dalmia,12000,nan,nan
Accounts,Mirza,25000,12000.000000,13000.000000
Accounts,Dravid,40000,25000.000000,15000.000000
Corporate,Karthikeyan,40000,nan,nan
Corporate,Bacchan,50000,40000.000000,10000.000000
Corporate,Ganesan,60000,50000.000000,10000.000000
Production,Dhoni,9000,nan,nan
Production,Ganguly,20000,9000.000000,11000.000000
Production,Mukherjee,25000,20000.000000,5000.000000
Sales,Dikshit,10000,nan,nan


#Transpose / Pivot
https://www.sqlshack.com/how-to-use-the-crosstab-function-in-postgresql/

In [82]:
!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c "create extension tablefunc;"

CREATE EXTENSION


In [83]:
runSelect('select * from dept')

deptid,deptname,managerid,location,locationid
10,Corporate,299034,Calcutta,1
20,Sales,239456,Calcutta,1
30,Accounts,822134,Calcutta,1
40,Production,349870,Bombay,2
60,Research,546224,Delhi,3


In [84]:
sql = "\
select * from \
crosstab('select location::text,locationid::text,deptname::text from dept order by 1,2') as locations \
(location text, dep1 text, dep2 text, dep3 text, dep4 text)\
"
runSelect(sql)

location,dep1,dep2,dep3,dep4
Bombay,Production,None,None,None
Calcutta,Corporate,Sales,Accounts,None
Delhi,Research,None,None,None


# Insert | Delete | Update

In [85]:
runSelect("select * from dept")

deptid,deptname,managerid,location,locationid
10,Corporate,299034,Calcutta,1
20,Sales,239456,Calcutta,1
30,Accounts,822134,Calcutta,1
40,Production,349870,Bombay,2
60,Research,546224,Delhi,3


In [86]:
runCmd("insert into Dept (DeptID, DeptName, ManagerID, Location,LocationID) VALUES \
	('11', 'Corporate2', '299035', 'Calcutta','1')")
runSelect("select * from dept")

Command executed


deptid,deptname,managerid,location,locationid
10,Corporate,299034,Calcutta,1
20,Sales,239456,Calcutta,1
30,Accounts,822134,Calcutta,1
40,Production,349870,Bombay,2
60,Research,546224,Delhi,3
11,Corporate2,299035,Calcutta,1


In [87]:
runCmd("update dept set deptname = 'Marketing' where deptid = '11'")
runSelect("select * from dept")

Command executed


deptid,deptname,managerid,location,locationid
10,Corporate,299034,Calcutta,1
20,Sales,239456,Calcutta,1
30,Accounts,822134,Calcutta,1
40,Production,349870,Bombay,2
60,Research,546224,Delhi,3
11,Marketing,299035,Calcutta,1


In [88]:
runCmd("delete from dept where deptid = '11'")
runSelect("select * from dept")

Command executed


deptid,deptname,managerid,location,locationid
10,Corporate,299034,Calcutta,1
20,Sales,239456,Calcutta,1
30,Accounts,822134,Calcutta,1
40,Production,349870,Bombay,2
60,Research,546224,Delhi,3


In [89]:
from datetime import datetime
import pytz
print('Last tested  ',datetime.now(pytz.timezone('Asia/Kolkata')))

Last tested   2024-11-05 14:25:35.613094+05:30


#ChronoTYM <br>
In Indic phraseology, tantra is knowledge, a yantra is a device to explore this knowledge and a mantra is the code that unlocks both the tantra and its yantra. Chrono/TYM a sci-fi trilogy that recreates this framework where the first novel Chronotantra defines the contours of a tech driven, futuristic society, the second, Chronoyantra, describes a device used to explore the genesis of this society, while the third, Chronomantra finally reveals the code, the key, that unlocks this puzzle.  [More information](http://bit.ly/chronobooks) <br>
![alt text](https://blogger.googleusercontent.com/img/b/R29vZ2xl/AVvXsEjdu_ajYwBRzk5iFYlcs59ZoRMMg6rhNm4JERKspvFJoveJsnfwRlzMICiC37UOja_Bd4sbfLA1pyyBbe0mj3tUfDBXoqCzNUDw3Wux7acoJBaLkWjAvJXfa7drRY2GQRAlFxEY9O7a7raY4RKy7cA3Uxgf5QsaZQ99C9GfXxFSex_rke8ROhAn41t8zOqK/s1640/tymBanner01_3.png)